In [44]:
from __future__ import division 
from pyomo.environ import * 
from gurobipy import*
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [45]:
m = AbstractModel('TFG_MMP_v1.0_February_2023')

In [46]:
m.G = Set()
m.p = Set()
m.s = Set() 
m.n = Set()
m.t = Set()
m.h = Set()
m.hf = Set()

In [47]:
data = DataPortal()
data.load(filename= 'g.csv', set= m.G)
data.load(filename= 'p.csv', set= m.p)
data.load(filename= 's.csv', set= m.s)
data.load(filename= 'n.csv', set= m.n)
data.load(filename= 't.csv', set= m.t)
data.load(filename= 'h.csv', set= m.h)
data.load(filename= 'hf.csv', set= m.hf)

In [48]:
m.e = 480

In [49]:

m.CO2rate = Param(m.G, within = NonNegativeReals)
m.alfa = Param(m.G, within = NonNegativeReals)
m.beta = Param(m.G, within = NonNegativeReals)
m.gamma = Param(m.G, within = NonNegativeReals)
m.f = Param(m.G, within = NonNegativeReals)
m.o = Param(m.G, within = NonNegativeReals)
m.qmax = Param(m.G, within = NonNegativeReals)
m.qmin = Param(m.G, within = NonNegativeReals)
m.bmax = Param(m.G, within = NonNegativeReals)
m.wmax = Param(m.G)
m.w0 = Param(m.G)
m.wmin = Param(m.G)
m.k = Param(m.G)
m.rend = Param(m.G)

# Read CSV file into pandas DataFrame and convert to Pyomo parameter
param_data = pd.read_csv('Data_duration_demand1.csv', header=0, index_col=[0, 1, 2])
m.a = Param(m.n, m.s, m.p, initialize=param_data['a'].to_dict())
m.d = Param(m.n, m.s, m.p, initialize=param_data['d'].to_dict())

param_data2 = pd.read_csv('Data_inflows1.csv', header=0, index_col=[0, 1])
m.i = Param(m.h, m.p, initialize=param_data2['i'].to_dict())


param_data3 = pd.read_csv('Data_qred1.csv', header=0, index_col=[0, 1, 2, 3])
m.qred = Param(m.t, m.s, m.n, m.p, initialize=param_data3['qred'].to_dict())

data.load(filename="Data_generators1.csv", param=(m.CO2rate, m.alfa , m.beta , m.gamma , m.f , m.o , m.qmax , m.qmin , m.bmax , m.wmax , m.w0 , m.wmin , m.k , m.rend))

instance = m.create_instance(data)

In [50]:
m.u = Var(m.t, m.s, m.p,within= Binary) #u(t,s,p) Decision to connect generator g
m.y = Var(m.t,m.s, m.p, within= Binary)#y(t,s,p) Start up decision for generator g
#m.z = Var(m.G, within= Binary)#z(p,s,g) Shut down decision for generator g #IT IS NOT USED IN THE MODEL

TypeError: __init__() takes exactly 3 positional arguments (4 given)

In [51]:
m.qt = Var(m.t, m.n, m.s, m.p) #q(t,n,s,p) Net power delivered by thermal generator t [GW]
m.qh = Var(m.h, m.n, m.s, m.p) #q(h,n,s,p) Net power delivered by hydraulic generator h [GW]
m.b = Var(m.h, m.n, m.s, m.p) #b(h,n,s,p)Gross power consumed by pumping-storage plant [GW]
m.w = Var(m.h, m.p) #w(h,p) Hydroelectric energy in the reservoir of the plant [GWh]

TypeError: __init__() takes exactly 3 positional arguments (5 given)

In [36]:
def obj_rule (m):
    return sum((m.f[t]*(m.gamma[t]*m.y[t,s,p] + sum((m.a[n, s, p] * (m.alfa[t] * m.qt[t,n,s,p] / m.k [t] + m.beta[t] * m.u[t, s, p])) for n in m.n))  + 
        sum((m.a [n, s, p]* m.o[t] * m.qt[t, n, s, p] / m.k[t]) for n in m.n )) for t in m.t for p in m.p for s in m.s)

m.obj = Objective(rule= obj_rule, sense= minimize)

In [37]:
#DECLARATION OF CONSTRAINTS (meaning of warnings????)
    #Demand balance and marginal cost
def Demand_Bal_rule(m,n,s,p):
    return sum((m.qt[t,n,s,p]) for t in m.t) + sum((m.qh[h,n,s,p] - m.b[h,n,s,p]) for h in m.h) == m.d[n,s,p]

m.Demand_Bal = Constraint(m.n, m.s, m.p, rule= Demand_Bal_rule)


    #Upper thermal limit 
def Upper_Thermal_rule(m,t,n,s,p):
    return m.qt[t,n,s,p] <= m.u[t,n,s,p] * m.k[t] * m.qmax[t]

m.Upper_Thermal = Constraint(m.t, m.n, m.s, m.p, rule= Upper_Thermal_rule)
    #Upper hydro constraints
def Upper_Hydro_rule(m,h,n,s,p):
    return m.qh[h,n,s,p] <= m.k[h] * m.qmax[h]

m.Upper_Hydro = Constraint(m.h, m.n, m.s, m.p, rule= Upper_Hydro_rule)
    #Maximum Capacity limits for pumped storage
def Cap_Hydro_rule(m,h,n,s,p):
    return m.b[h,n,s,p] <= m.k[h] * m.bmax[h]

m.Cap_Hydro = Constraint(m.h, m.n, m.s, m.p, rule= Cap_Hydro_rule)
    #Lower thermal limit
def Lower_Thermal_rule(m,t,n,s,p):
    return m.qt[t,n,s,p] >= m.u[t,n,s,p] * m.k[t] * m.qmin[t]

m.Lower_Thermal = Constraint(m.t, m.n, m.s, m.p, rule= Lower_Thermal_rule)
    #Energy balance in an equivalent reservoir
def E_Bal_rule(m,h,p):
    return m.w[h,p] + sum(m.a[n,s,p] * (m.qh[h,n,s,p] - m.rend[h] * m.b[h,n,s,p]) for n in m.n for s in m.s) <= m.w[h,p - 1] + m.i[h,p]

m.E_Bal = Constraint(m.h, m.p, rule= E_Bal_rule)
    #Upper limit equivalent energy stored in reservoir
def Upper_W_rule(m,h,p):
    return m.w[h,p] <= m.wmax[h,p]

m.Upper_W = Constraint(m.h, m.p, rule = Upper_W_rule)
    #Lower limit equivalent energy stored in basin
def Lower_W_rule(m,h,p):
    return m.w[h,p] >= m.wmin[h,p]

m.Lower_W = Constraint(m.h, m.p, rule = Lower_W_rule)
    #Minimum operating hours during the year
def Min_H_rule(m,t):
    return sum((m.a[n,s,p] * m.qt[t,n,s,p]) for n in m.n for s in m.s for p in m.p) >= m.k[t] * m.qmax[t] * m.e

m.Min_H = Constraint(m.t, rule = Min_H_rule)
    #Minimum production to accommodate grid constraints
def Min_Grid_rule(m,t,n,s,p):
    return m.qt[t,n,s,p] >= m.qred[t,n,s,p]

m.Min_Grid = Constraint(m.t, m.n, m.s, m.p, rule = Min_Grid_rule)

In [38]:
workPath = os.getcwd()

In [39]:
print(data['G'],'\n')
print(data['p'],'\n')
print(data['s'],'\n')
print(data['n'],'\n')
print(data['t'],'\n')
print(data['h'],'\n')
print(data['hf'],'\n')

['GEN_001', 'GEN_002', 'GEN_003', 'GEN_004', 'GEN_005', 'GEN_006', 'GEN_007', 'GEN_008', 'HYDRO_RES', 'HYDRO_ROR', 'HYDRO_PUM'] 

['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'p11', 'p12'] 

['lab', 'fes'] 

['n1', 'n2', 'n3', 'n4', 'n5'] 

['GEN_001', 'GEN_002', 'GEN_003', 'GEN_004', 'GEN_005', 'GEN_006', 'GEN_007', 'GEN_008'] 

['HYDRO_RES', 'HYDRO_ROR', 'HYDRO_PUM'] 

['HYDRO_PUM'] 



In [40]:

instance.pprint()

11 Set Declarations
    G : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {'GEN_001', 'GEN_002', 'GEN_003', 'GEN_004', 'GEN_005', 'GEN_006', 'GEN_007', 'GEN_008', 'HYDRO_RES', 'HYDRO_ROR', 'HYDRO_PUM'}
    a_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     3 :  n*s*p :  120 : {('n1', 'lab', 'p1'), ('n1', 'lab', 'p2'), ('n1', 'lab', 'p3'), ('n1', 'lab', 'p4'), ('n1', 'lab', 'p5'), ('n1', 'lab', 'p6'), ('n1', 'lab', 'p7'), ('n1', 'lab', 'p8'), ('n1', 'lab', 'p9'), ('n1', 'lab', 'p10'), ('n1', 'lab', 'p11'), ('n1', 'lab', 'p12'), ('n1', 'fes', 'p1'), ('n1', 'fes', 'p2'), ('n1', 'fes', 'p3'), ('n1', 'fes', 'p4'), ('n1', 'fes', 'p5'), ('n1', 'fes', 'p6'), ('n1', 'fes', 'p7'), ('n1', 'fes', 'p8'), ('n1', 'fes', 'p9'), ('n1', 'fes', 'p10'), ('n1', 'fes', 'p11'), ('n1', 'fes', 'p12'), ('n2', 'lab', 'p1'), ('n2', 'lab', 'p2'), ('n2', 'lab', 'p3'), ('n2', 'lab',

         ('GEN_008', 'fes', 'n4', 'p2') :   0.0
         ('GEN_008', 'fes', 'n4', 'p3') :   0.0
         ('GEN_008', 'fes', 'n4', 'p4') :   0.0
         ('GEN_008', 'fes', 'n4', 'p5') :   0.0
         ('GEN_008', 'fes', 'n4', 'p6') :   0.0
         ('GEN_008', 'fes', 'n4', 'p7') :   0.0
         ('GEN_008', 'fes', 'n4', 'p8') :   0.0
         ('GEN_008', 'fes', 'n4', 'p9') :   0.0
         ('GEN_008', 'fes', 'n5', 'p1') :   0.0
        ('GEN_008', 'fes', 'n5', 'p10') :   0.0
        ('GEN_008', 'fes', 'n5', 'p11') :   0.0
        ('GEN_008', 'fes', 'n5', 'p12') :   0.0
         ('GEN_008', 'fes', 'n5', 'p2') :   0.0
         ('GEN_008', 'fes', 'n5', 'p3') :   0.0
         ('GEN_008', 'fes', 'n5', 'p4') :   0.0
         ('GEN_008', 'fes', 'n5', 'p5') :   0.0
         ('GEN_008', 'fes', 'n5', 'p6') :   0.0
         ('GEN_008', 'fes', 'n5', 'p7') :   0.0
         ('GEN_008', 'fes', 'n5', 'p8') :   0.0
         ('GEN_008', 'fes', 'n5', 'p9') :   0.0
         ('GEN_008', 'lab', 'n1', 'p1') 

In [43]:
solver = SolverFactory('gurobi')
results = solver.solve(instance)
instance.pprint()
results.write()

ValueError: ERROR: No objectives defined for input model. Cannot write legal LP file.